<a href="https://colab.research.google.com/github/RhysWangJunfei/nilm/blob/master/AMPDs/training/nilm_rnn_woe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import tensorflow as tf
import numpy as np
import io
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split 
from google.colab import files
import matplotlib.pyplot as plt

In [0]:
'''Sliding window function'''
def create_dataset(dataset, look_back=1):
    dataX = []
    for i in range(len(dataset)-look_back+1):
        a = dataset[i:(i+look_back)]
        dataX.append(a)
    return np.array(dataX)

In [0]:
WHE_data = pd.read_csv('Electricity_WHE.csv')['P']
WOE_data = pd.read_csv('Electricity_WOE.csv')['P']

In [11]:
window_size=60

dataX_raw = create_dataset(WHE_data.as_matrix(), window_size)

app_Y_raw = WOE_data[window_size-1:].values.reshape([WOE_data.shape[0]-window_size+1,1])
dataX = np.concatenate([dataX_raw[0:472500,:],dataX_raw[475500:,:]],axis=0)
app_Y = np.concatenate([app_Y_raw[0:472500,:],app_Y_raw[475500:,:]],axis=0)
categorized_Y = np.ones(app_Y.shape)*1
categorized_Y[[np.where(app_Y<800)[0]],:]=0

encoder = OneHotEncoder(handle_unknown='ignore',sparse=False)
Y_1hot = encoder.fit_transform(categorized_Y)

dataX_train_cv = np.concatenate([dataX[0:450000,:],dataX[470000:,:]],axis=0)
dataX_test = dataX[450000:470000,:]
Y_1hot_train_cv = np.concatenate([Y_1hot[0:450000,:],Y_1hot[470000:,:]],axis=0)
Y_1hot_test = Y_1hot[450000:470000,:]

X_train, X_cv, y_train, y_cv = train_test_split(dataX_train_cv, Y_1hot_train_cv, test_size=0.01, shuffle=True)
scaler = MinMaxScaler()
scaler.fit(X_train.astype(float))
X_train = scaler.transform(X_train)
X_cv = scaler.transform(X_cv)
X_test = scaler.transform(dataX_test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
max_indice = np.argmax(Y_1hot,1)
df = pd.Series(max_indice)
df.value_counts()

0    1045658
1       2483
dtype: int64

In [0]:
'''Hyper parameters for deep learning'''
# Hyper Parameters
LR = 0.001               # learning rate
#cfg_list = nf.model_configs()
#error_list = []

#hyperparameters
batch_size=512
num_units = [256, 256]

'''RNN Model Definition'''
tf.reset_default_graph()
''''''
#define inputs
tf_x = tf.placeholder(tf.float32, [None, window_size,1],name='x')
tf_y = tf.placeholder(tf.int32, [None, 2],name='y')

cells = [tf.keras.layers.LSTMCell(units=n) for n in num_units]
stacked_rnn_cell = tf.keras.layers.StackedRNNCells(cells)
outputs, (h_c, h_n) = tf.nn.dynamic_rnn(
        stacked_rnn_cell,                   # cell you have chosen
        tf_x,                      # input
        initial_state=None,         # the initial hidden state
        dtype=tf.float32,           # must given if set initial_state = None
        time_major=False,           # False: (batch, time step, input); True: (time step, batch, input)
)
l1 = tf.layers.dense(outputs[:, -1, :],256,activation=tf.nn.leaky_relu,name='l1')
l2 = tf.layers.dense(l1,128,activation=tf.nn.leaky_relu,name='l2')
l3 = tf.layers.dense(l2,64,activation=tf.nn.leaky_relu,name='l3')
l4 = tf.layers.dense(l3,32,activation=tf.nn.leaky_relu,name='l4')
l5 = tf.layers.dense(l4,16,activation=tf.nn.leaky_relu,name='l5')
l6 = tf.layers.dense(l5,8,activation=tf.nn.leaky_relu,name='l6')
l7 = tf.layers.dense(l6,4,activation=tf.nn.leaky_relu,name='l7')
pred = tf.layers.dense(l4,2,activation=tf.nn.relu,name='pred')

with tf.name_scope('loss'):
    cross_entropy =  tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_y, logits=pred) 
    loss = tf.reduce_mean(cross_entropy)
    tf.summary.scalar("loss",tensor=loss)

train_op = tf.train.AdamOptimizer(LR).minimize(loss)

accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(tf_y, axis=1), tf.argmax(pred, axis=1)), tf.float32))

init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer()) 
saver = tf.train.Saver()

In [16]:
from imblearn.under_sampling import RandomUnderSampler
tl = RandomUnderSampler(sampling_strategy={0:2400,1:2400})

sess = tf.Session()
#sess.run(init_op)
saver.restore(sess, 'my_net/save_net_rnn.ckpt')

for i in range(0,1):
    X, y_1 = tl.fit_resample(X_train, y_train)
    y=encoder.transform(y_1)
    whole_train = np.concatenate([X,y],axis=1)
    for j in range(0,1):
        print('Loop'+str(i)+',###iteration: '+str(j)+'###')
        batch_index = np.random.choice(4800,batch_size)
        batch_train = whole_train[batch_index,:]
        batch_X = batch_train[:,:-2].reshape([batch_size,window_size,1])
        batch_y = batch_train[:,-2:]
        sess.run(train_op,{tf_x:batch_X , tf_y:batch_y})
        cost_ = sess.run(loss,{tf_x:batch_X, tf_y:batch_y})
        print('train loss= %.4f' % cost_)
        if(j%99==0):
            acc_train = sess.run(accuracy,{tf_x:batch_X, tf_y:batch_y})
            acc_cv = sess.run(accuracy,feed_dict={tf_x: X_cv.reshape([X_cv.shape[0],window_size,1]), tf_y:y_cv})
            print('train loss= %.4f' % cost_+', Acc=%.2f'% acc_train)
            print('Test Acc=%.2f'% acc_cv)
            pre = sess.run(pred,feed_dict={tf_x: X_cv.reshape([-1,window_size,1]), tf_y:y_cv})
            y_lables_argmax = tf.argmax(y_cv,axis=1)  
            y_pred_argmax = tf.argmax(pre,axis=1)
            confusion = tf.confusion_matrix(labels=y_lables_argmax, predictions=y_pred_argmax, num_classes=2)
            print(confusion.eval(session=sess,feed_dict={tf_x: batch_X, tf_y:batch_y}))
            save_path = saver.save(sess, "my_net/save_net_rnn.ckpt")
#pre = sess.run(pred,feed_dict={tf_x: X_test.reshape([X_test.shape[0],window_size,1]), tf_y: y_test})
#y_lables_argmax = np.argmax(y_test,1)
#y_pred_argmax = np.argmax(pre,1)
#confusion = tf.confusion_matrix(labels=y_lables_argmax, predictions=y_pred_argmax, num_classes=2)
#print('Confusion Matrix: \n\n', tf.Tensor.eval(confusion,feed_dict=None))
#print(confusion.eval(session=sess))

pre_test = sess.run(pred,feed_dict={tf_x: X_test.reshape([-1,window_size,1]), tf_y:Y_1hot_test})
ytest_lables_argmax = tf.argmax(Y_1hot_test,axis=1)  
ytest_pred_argmax = tf.argmax(pre_test,axis=1)
confusion = tf.confusion_matrix(labels=ytest_lables_argmax, predictions=ytest_pred_argmax, num_classes=2)
print(confusion.eval(session=sess,feed_dict={tf_x: batch_X, tf_y:batch_y}))
sess.close()

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from my_net/save_net_rnn.ckpt
Loop0,###iteration: 0###
train loss= 0.0706
train loss= 0.0706, Acc=0.98
Test Acc=0.96
[[9882  378]
 [   0   22]]
[[18766  1185]
 [    0    49]]
